In [1]:
import scanpy as sc
import omicverse as ov

ov.plot_set()

ModuleNotFoundError: No module named 'omicverse'

In [ ]:
# 定义您的分组列和细胞类型注释列名称
GROUP_COL = "group"  # 替换为您的实际分组列名，例如 'ctrl' 和 'stim'
SIMPLE_COL = "patients_organ"


In [ ]:
# 设置参数
import matplotlib as mpl

mpl.rcParams["pdf.fonttype"] = 42  # 保留字体
sc.settings.verbosity = 4  # 输出细节
sc._settings.ScanpyConfig.n_jobs = -1  # 使用所有核心
sc.settings.set_figure_params(
    dpi=80,
    dpi_save=600,
    facecolor="white",
    frameon=False,  # remove frame
)

In [ ]:
adata = sc.read("results/adata_raw_CCL_LCL/anndata_annotation_harmony_celltypist.h5ad")

In [ ]:
%%time
adata=ov.pp.preprocess(adata,mode='shiftlog|pearson',n_HVGs=2000,)
adata

In [ ]:
ov.pp.scale(adata)
ov.pp.pca(adata)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import patheffects

fig, ax = plt.subplots(figsize=(4, 4))
ov.pl.embedding(
    adata,
    basis="X_umap",
    color=[SIMPLE_COL],
    frameon="small",
    title="Celltypes",
    # legend_loc='on data',
    legend_fontsize=14,
    legend_fontoutline=2,
    # size=10,
    ax=ax,
    # legend_loc=True,
    add_outline=False,
    # add_outline=True,
    outline_color="black",
    outline_width=1,
    show=False,
)

- 初始化模型

In [ ]:
import numpy as np
## Initialize the cnmf object that will be used to run analyses
cnmf_obj = ov.single.cNMF(adata,components=np.arange(3,20), n_iter=200, seed=123, num_highvar_genes=2000,
                          output_dir='example_dg/cNMF', name='dg_cNMF')

In [ ]:
## Specify that the jobs are being distributed over a single worker (total_workers=1) and then launch that worker
cnmf_obj.factorize(worker_i=0, total_workers=20)

In [ ]:
cnmf_obj.combine(skip_missing_files=True)

- Compute the stability and error at each choice of K to see if a clear choice jumps out

In [ ]:
cnmf_obj.k_selection_plot(close_fig=False)

In this range, K=7 gave the most stable solution so we will begin by looking at that.

The next step computes the consensus solution for a given choice of K. We first run it without any outlier filtering to see what that looks like. Setting the density threshold to anything >= 2.00 (the maximum possible distance between two unit vectors) ensures that nothing will be filtered.

In [14]:
selected_K = 7
density_threshold = 2.00

In [ ]:
cnmf_obj.consensus(
    k=selected_K,
    density_threshold=density_threshold,
    show_clustering=True,
    close_clustergram_fig=False,
)

In [18]:
density_threshold = 0.10

In [ ]:

cnmf_obj.consensus(
    k=selected_K,
    density_threshold=density_threshold,
    show_clustering=True,
    close_clustergram_fig=False,
)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import patheffects

from matplotlib import gridspec
import matplotlib.pyplot as plt

width_ratios = [0.2, 4, 0.5, 10, 1]
height_ratios = [0.2, 4]
fig = plt.figure(figsize=(sum(width_ratios), sum(height_ratios)))
gs = gridspec.GridSpec(
    len(height_ratios),
    len(width_ratios),
    fig,
    0.01,
    0.01,
    0.98,
    0.98,
    height_ratios=height_ratios,
    width_ratios=width_ratios,
    wspace=0,
    hspace=0,
)

D = cnmf_obj.topic_dist[cnmf_obj.spectra_order, :][:, cnmf_obj.spectra_order]
dist_ax = fig.add_subplot(
    gs[1, 1],
    xscale="linear",
    yscale="linear",
    xticks=[],
    yticks=[],
    xlabel="",
    ylabel="",
    frameon=True,
)
dist_im = dist_ax.imshow(
    D, interpolation="none", cmap="viridis", aspect="auto", rasterized=True
)

left_ax = fig.add_subplot(
    gs[1, 0],
    xscale="linear",
    yscale="linear",
    xticks=[],
    yticks=[],
    xlabel="",
    ylabel="",
    frameon=True,
)
left_ax.imshow(
    cnmf_obj.kmeans_cluster_labels.values[cnmf_obj.spectra_order].reshape(-1, 1),
    interpolation="none",
    cmap="Spectral",
    aspect="auto",
    rasterized=True,
)

top_ax = fig.add_subplot(
    gs[0, 1],
    xscale="linear",
    yscale="linear",
    xticks=[],
    yticks=[],
    xlabel="",
    ylabel="",
    frameon=True,
)
top_ax.imshow(
    cnmf_obj.kmeans_cluster_labels.values[cnmf_obj.spectra_order].reshape(1, -1),
    interpolation="none",
    cmap="Spectral",
    aspect="auto",
    rasterized=True,
)

cbar_gs = gridspec.GridSpecFromSubplotSpec(
    3, 3, subplot_spec=gs[1, 2], wspace=0, hspace=0
)
cbar_ax = fig.add_subplot(
    cbar_gs[1, 2],
    xscale="linear",
    yscale="linear",
    xlabel="",
    ylabel="",
    frameon=True,
    title="Euclidean\nDistance",
)
cbar_ax.set_title("Euclidean\nDistance", fontsize=12)
vmin = D.min().min()
vmax = D.max().max()
fig.colorbar(
    dist_im,
    cax=cbar_ax,
    ticks=np.linspace(vmin, vmax, 3),
)
cbar_ax.set_yticklabels(cbar_ax.get_yticklabels(), fontsize=12)
plt.tight_layout()  # 确保布局紧凑
plt.savefig("./figures/31-亚群分析-NMF聚类图.png")
plt.show()

In [ ]:
density_filter = cnmf_obj.local_density.iloc[:, 0] < density_threshold
fig, hist_ax = plt.subplots(figsize=(4, 4))

# hist_ax = fig.add_subplot(hist_gs[0,0], xscale='linear', yscale='linear',
#   xlabel='', ylabel='', frameon=True, title='Local density histogram')
hist_ax.hist(cnmf_obj.local_density.values, bins=np.linspace(0, 1, 50))
hist_ax.yaxis.tick_right()

xlim = hist_ax.get_xlim()
ylim = hist_ax.get_ylim()
if density_threshold < xlim[1]:
    hist_ax.axvline(density_threshold, linestyle="--", color="k")
    hist_ax.text(
        density_threshold + 0.02, ylim[1] * 0.95, "filtering\nthreshold\n\n", va="top"
    )
hist_ax.set_xlim(xlim)
hist_ax.set_xlabel(
    "Mean distance to k nearest neighbors\n\n%d/%d (%.0f%%) spectra above threshold\nwere removed prior to clustering"
    % (sum(~density_filter), len(density_filter), 100 * (~density_filter).mean())
)
hist_ax.set_title("Local density histogram")

In [ ]:
result_dict = cnmf_obj.load_results(
    K=selected_K, 
    density_threshold=density_threshold,
    # n_top_genes=None,
    n_top_genes=100,
                                    )
result_dict["usage_norm"].head()

In [ ]:
result_dict["gep_scores"].head()

In [ ]:
result_dict["gep_tpm"].head()

In [ ]:
result_dict["top_genes"].to_csv("./table/31-NMFgene_谱评分_top100.csv")
result_dict["top_genes"]

In [ ]:
cnmf_obj.get_results(adata, result_dict)

In [ ]:
ov.pl.embedding(
    adata,
    basis="X_umap",
    color=result_dict["usage_norm"].columns,
    use_raw=False,
    ncols=3,
    vmin=0,
    vmax=1,
    frameon="small",
)

- 直接用，细胞类型重叠

In [ ]:
cnmf_obj.get_results_rfc(
    adata, result_dict, use_rep="scaled|original|X_pca", cNMF_threshold=0.5
)

In [ ]:
ov.pl.embedding(
    adata,
    basis="X_umap",
    color=["cNMF_cluster_rfc", "cNMF_cluster_clf"],
    frameon="small",
    # title="Celltypes",
    # legend_loc='on data',
    legend_fontsize=14,
    legend_fontoutline=2,
    # size=10,
    # legend_loc=True,
    add_outline=False,
    # add_outline=True,
    outline_color="black",
    outline_width=1,
    show=False,
)

In [43]:
plot_genes = []
for i in result_dict["top_genes"].columns:
    plot_genes += result_dict["top_genes"][i][:5].values.reshape(-1).tolist()

In [50]:
adata.obs["cNMF_cluster"] = adata.obs["cNMF_cluster_rfc"].apply(lambda x: "cNMF_" + x)

In [ ]:
ov.pl.embedding(
    adata,
    basis="X_umap",
    color=["cNMF_cluster"],
    frameon="small",
    # title="Celltypes",
    # legend_loc='on data',
    legend_fontsize=14,
    legend_fontoutline=2,
    # size=10,
    # legend_loc=True,
    add_outline=False,
    # add_outline=True,
    outline_color="black",
    outline_width=1,
    show=False,
    save="-cNMF-聚类图.pdf"
)

In [ ]:
sc.pl.dotplot(
    adata,
    plot_genes,
    "cNMF_cluster",
    dendrogram=False,
    log=True,
    standard_scale="var",
)

In [52]:
adata.write("sub_anndata_cnmf.h5ad")

In [3]:
adata = sc.read("sub_anndata_cnmf.h5ad")

In [ ]:
GROUP_BY = "cNMF_cluster"

sc.tl.rank_genes_groups(
    adata,
    groupby=GROUP_BY,
    method="t-test",
    # use_raw=True,
    layer="lognorm",
)

In [ ]:
sc.pl.rank_genes_groups(adata, ncols=4)

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata,
    n_genes=5,
    # vmin=0,
    # vmax=20,
    save="-31-亚群分析-NMF-marker可视化-点图.pdf",
)

In [ ]:
sc.pl.rank_genes_groups_matrixplot(
    adata,
    n_genes=5,
    standard_scale="var",
    use_raw=False,
    save="-31-亚群分析-NMF-marker可视化-热图.pdf",
)

In [ ]:
sc.tl.filter_rank_genes_groups(
    adata,
    min_fold_change=1,  # 最小折叠变化阈值
    min_in_group_fraction=0.25,  # 在组内的最小基因表达比例
    max_out_group_fraction=0.5,  # 在组外的最大基因表达比例
    key="rank_genes_groups",  # 基因组数据的键
    key_added="rank_genes_groups_filtered",  # 过滤后的基因组数据的键
    use_raw=False,
)

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata,
    groupby=GROUP_BY,
    standard_scale="var",
    n_genes=5,
    key="rank_genes_groups_filtered",
    save = "-31-亚群分析-NMF-marker可视化-点图-过滤.pdf"
    use_raw=False,
)

In [ ]:
sc.pl.rank_genes_groups_matrixplot(
    adata,
    n_genes=5,
    standard_scale="var",
    key="rank_genes_groups_filtered",
    use_raw=False,
    save = "-31-亚群分析-NMF-marker可视化-热图-过滤.pdf"
)

In [11]:
##  保存全部maker列表

deg_table = sc.get.rank_genes_groups_df(
    adata,
    group=adata.obs[GROUP_BY].unique(),
    key="rank_genes_groups",
)
deg_table = deg_table.dropna(subset=["names"])
deg_table[:5]
deg_table.to_csv("table/21-all_rank_genes.csv", index=False)

In [ ]:
##  保存全部maker列表-过滤后的

deg_table = sc.get.rank_genes_groups_df(
    adata,
    group=adata.obs[GROUP_BY].unique(),
    key="rank_genes_groups_filtered",
)
deg_table = deg_table.dropna(subset=["names"])
deg_table.to_csv("table/21-all_rank_genes_groups_filtered.csv", index=False)
deg_table[:5]

## 看看在那个亚群咋们的bulkgene最牛


In [ ]:
mu_gene = ["Fez2", "Fez1", "Atp2a2", "Atp6v1a", "Gabarap", "Kdr"]

sc.pl.tracksplot(
    # adata[adata.obs["group"] == "Disorder"],
    adata,
    mu_gene,
    groupby="cNMF_cluster",
    # dendrogram = True,
    save="-26-BULK-gene可视化-亚群表达线热图.pdf",
    figsize=(15, 5),
)

## 免疫因子富集分析

In [ ]:
import pandas as pd

sheet_dict = pd.read_excel(
    "../data/极化分析/41586_2023_6816_MOESM5_ESM.xlsx", sheet_name=None
)
cytokine_responses = pd.concat(sheet_dict.values()).reset_index(drop=True)

cytokine_responses_cell = "Macrophage"

celltype_response = cytokine_responses.query(
    f"Celltype_Str == '{cytokine_responses_cell}'"
).copy()

celltype_response["Gene"] = celltype_response["Gene"].map(lambda s: [s])
response_sets = celltype_response.groupby(["Cytokine"])["Gene"].sum().to_dict()
print(response_sets)

In [ ]:
SUB_TYPE = "cNMF_6"
rank_gene = deg_table.loc[deg_table[GROUP_COL] == SUB_TYPE, :].iloc[:, 1:3]
rank_gene.columns = ["gene_name","rnk"]
rank_gene.head()

In [138]:
gsea_obj = ov.bulk.pyGSEA(rank_gene, response_sets, cutoff=1,outdir=f"./enrichr_gsea-{SUB_TYPE}")

In [ ]:
response_sets.keys()

In [140]:
rank_gene["rnk"] = rank_gene["rnk"].astype(float)

In [ ]:
enrich_res = gsea_obj.enrichment()

In [ ]:
import pandas as pd

res_2d = pd.read_csv(f"./enrichr_gsea-{SUB_TYPE}/gseapy.gene_set.prerank.report.csv")
res_2d

In [143]:
import networkx as nx
import gseapy as gp

In [ ]:
nodes, edges = gp.enrichment_map(res_2d)  # 默认按照Adjusted P-value过滤0.05

In [ ]:
G = nx.from_pandas_edgelist(
    edges,
    source="src_idx",
    target="targ_idx",
    edge_attr=["jaccard_coef", "overlap_coef", "overlap_genes"],
)

# Add missing node if there is any
for node in nodes.index:
    if node not in G.nodes():
        G.add_node(node)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8, 8))

# init node cooridnates
pos = nx.layout.spiral_layout(G)
# pos = nx.layout.spring_layout(G, seed=42)
# node_size = nx.get_node_attributes()
# draw node
nx.draw_networkx_nodes(
    G,
    pos=pos,
    cmap=plt.cm.RdYlBu,
    node_color=list(nodes.NES),
    node_size=list(nodes.Hits_ratio * 1000),
    alpha=0.5,
)
# draw node label
nx.draw_networkx_labels(G, pos=pos, labels=nodes.Term.to_dict())
# draw edge
edge_weight = nx.get_edge_attributes(G, "jaccard_coef").values()
nx.draw_networkx_edges(
    G, pos=pos, width=list(map(lambda x: x * 10, edge_weight)), edge_color="green",alpha=0.2
)
# 去掉坐标轴
plt.axis("off")
plt.savefig(f"./figures/31-亚群分析-NMF-免疫GSEA富集分析-网络图-{SUB_TYPE}.pdf")
plt.show()